#### **Evidence for model**
$$\log P(\{\log L_i \} \mid \bm{\theta}) \propto \log P(\bm{\theta} \mid \{\log L_i \}) \approx
\log P'(\bm{\theta}) = \log P'(\bm{\theta}_0) - \frac{1}{2} (\bm{\theta} - \bm{\theta}_0)^\intercal \bm{A} (\bm{\theta} - \bm{\theta}_0)$$
$$ \log Z = \log L_\mathrm{max} + \frac{1}{2} \log |2\pi\Sigma| - \log V_\pi $$
$$ \log Z = \log P'(\bm{\theta}_0)  + \frac{1}{2} \log |2\pi H^{-1}| - \log V_\pi $$
$$ Z = P'(\bm{\theta}_0) \sqrt{\frac{2\pi |H^{-1}|}{V_\pi}} $$
$$ \log Z \sim \log P'(\bm{\theta}_0) - \frac{1}{2}\log|H| $$

Remember $ \log P(\bm{\theta} \mid \{\log L_i \}$ and hence $\log P'(\bm{\theta}_0)$ has a normalising factor $-\frac{1}{2} \log |\Sigma| = +\frac{1}{2} \log |\Sigma^{-1}|$ in front

In [1]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.optimize import minimize, least_squares

# Load test samples
from aeons.tools import pickle_in
samples_g_1, samples_c_1, samples_w_1 = pickle_in("../test_samples/samples_1.pickle")

from aeons.covariance import logX_mu, logX_Sigmainv, points_at_iteration, X_mu, X_Sigma, X_Sigmainv, data_at_iteration
from aeons.true_distribution import generate_Xs
from aeons.lm_partial import analytic_lm_params
from aeons.bayes import logPr_bayes, logPr_laplace, logPr_gaussian, minimise_ls, minimise_bayes, minimise_gaussian
from aeons.likelihoods import likelihood, linear_like, quad_like, log_like, simple_like, middle_like, full_like
linear, quad, log = linear_like(), quad_like(), log_like()
simple, middle, full = simple_like(), middle_like(), full_like()
simple_log, middle_log, full_log = simple_like(logX=True), middle_like(logX=True), full_like(logX=True)
from aeons.hessian import hess_autograd

In [2]:
def full_log_like():
    def func(X, theta):
        logLmax, d, sigma = theta
        X = np.exp(X)
        return logLmax - X**(2/d)/(2*sigma**2)
    def inverse(logL, theta):
        logLmax, d, sigma = theta
        return np.log((2*sigma**2 * (logLmax - logL))**(d/2))
    def prime(X, theta):
        logLmax, d, sigma = theta
        X = np.exp(X)
        return - 1/(d*sigma**2) * X**(2/d)
    return likelihood(func, inverse, prime)
full_log = full_log_like()

In [3]:
import torch
def full_log_like_torch():
    def func(X, theta):
        logLmax, d, sigma = theta
        X = torch.exp(X)
        return logLmax - X**(2/d)/(2*sigma**2)
    def inverse(logL, theta):
        logLmax, d, sigma = theta
        logL = torch.tensor(logL)
        return torch.log((2*sigma**2 * (logLmax - logL))**(d/2))
    def prime(X, theta):
        logLmax, d, sigma = theta
        X = torch.exp(torch.tensor(X))
        return - 1/(d*sigma**2) * X**(2/d)
    return likelihood(func, inverse, prime)
full_log_torch = full_log_like_torch()

def logPr_bayes_torch(logL, likelihood, mean, covinv, theta):
    """likelihood = f(X_i, theta)"""
    Xstar = likelihood.inverse(logL, theta)
    log_abs_fprimes = torch.log(abs(likelihood.prime(Xstar, theta)))
    return - torch.sum(log_abs_fprimes)- 1/2 * (Xstar - mean).T @ covinv @ (Xstar - mean)

def minimise_bayes(logL, likelihood, mean, covinv, x0):
    def func(theta):
        return - logPr_bayes(logL, likelihood, mean, covinv, theta)
    solution = minimize(func, x0, method='Nelder-Mead')
    return solution

In [4]:
def logZ(y, likelihood, mean, covinv, theta_max, H):
    eigvals_covinv = np.linalg.eigvals(covinv)
    logPr_max = logPr_bayes(y, likelihood, mean, covinv, theta_max) + 1/2 * np.log(eigvals_covinv).sum()
    return logPr_max - 1/2 * np.log(abs(np.linalg.det(H)))

In [10]:
nk = 500 * np.ones(1000)
mean_X = X_mu(nk)
covinv_X = X_Sigmainv(nk)
mean_logX = logX_mu(nk)
covinv_logX = logX_Sigmainv(nk)

In [6]:
# # nk, L = data_at_iteration(samples_w_1, 1000)
# mean = X_mu(nk)
# covinv = X_Sigmainv(nk)
# theta = minimise_bayes(L, full, mean, covinv, [1, 10, 0.1]).x

/home/zixiao/Documents/III/project/aeons/aeons/likelihoods.py:111: RuntimeWarning: divide by zero encountered in double_scalars
  return - 1/(d*sigma**2) * X**(2/d - 1)


In [15]:
Xs = generate_Xs(nk)
theta_true = np.array([1, 10, 0.1])
y = full.func(Xs, theta_true)
theta_logX = minimise_bayes(y, full_log, mean_logX, covinv_logX, theta_true).x
theta_X = minimise_bayes(y, full, mean_X, covinv_X, theta_true).x
X_X = full.inverse(y, theta_X)
X_logX = full.inverse(y, theta_logX)
H_X = hess_autograd(y, full, mean_X, covinv_X, theta_X)
H_logX = hess_autograd(y, full_log_torch, mean_logX, covinv_logX, theta_logX)
ratio = np.exp(logZ(y, full, mean_X, covinv_X, theta_X, H_X) - logZ(y, full_log, mean_logX, covinv_logX, theta_logX, H_logX))
print(f'X: {theta_X}')
print(f'logX: {theta_logX}')
print(f'bayes factor: {ratio}')

X: [ 0.98336328 10.78177846  0.10000351]
logX: [ 0.12138    10.69683441  0.1008769 ]
bayes factor: 1.7396874238412947


In [18]:
ratio = np.exp(logZ(y, full, mean_X, covinv_X, theta_X, H_X) - logZ(y, full, mean_X, covinv_X, [2, 12, 0.8], H_X))

/tmp/ipykernel_1592191/333922539.py:1: RuntimeWarning: overflow encountered in exp
  ratio = np.exp(logZ(y, full, mean_X, covinv_X, theta_X, H_X) - logZ(y, full, mean_X, covinv_X, [2, 12, 0.8], H_X))


In [17]:
from ipywidgets import interact, widgets
@interact(frame=widgets.RadioButtons(options=['full', 1, 2, 3]))
def plot(frame):
    plt.plot(Xs, y, 'x', ms=1, color='deepskyblue')
    plt.plot(X_X, y, color='black', label='X')
    plt.plot(X_logX, y, color='gray', label='logX')
    if frame == 'full':
        pass
    elif frame == 1:
        plt.xlim(0.2, 0.4)
        plt.ylim(-40, -35)
    elif frame == 2:
        plt.xlim(0.4, 0.46)
        plt.ylim(-42, -40)
    else:
        plt.xlim(0.6, 0.8)
        plt.ylim(-47.5, -43)
    plt.legend();

interactive(children=(RadioButtons(description='frame', options=('full', 1, 2, 3), value='full'), Output()), _…